# Tidy Data Exercises

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from pydataset import data
from env import get_db_url
import os
import matplotlib.pyplot as plt

1. Attendance Data

- Read the data from the attendance table and calculate an attendance percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [3]:
#pull the attendance dataset from the database
df = pd.read_sql('SELECT * FROM attendance', get_db_url('tidy_data'))
#view the pulled dataset
df

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [9]:
#view the column names
df.columns

Index(['Unnamed: 0', '2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
       '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08'],
      dtype='object')

In [16]:
#rename the 'unnamed' column
df.rename(columns={'Unnamed: 0': 'student'}, inplace=True)
df

,student,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [17]:
#melt the df so it is workable
melt_df = df.melt(id_vars=['student'], var_name='dates', value_name='attendence')
melt_df

,student,dates,attendence
0,Sally,2018-01-01,P
1,Jane,2018-01-01,A
2,Billy,2018-01-01,A
3,John,2018-01-01,P
4,Sally,2018-01-02,T
5,Jane,2018-01-02,P
6,Billy,2018-01-02,T
7,John,2018-01-02,T
8,Sally,2018-01-03,T
9,Jane,2018-01-03,T


In [31]:
#assign values to the attendance strings based on the attendance credit recieved and change to float
melt_df['attnd'] = melt_df.attendence.replace({'A': '0', 'T': '0.90', 'H': '0.5', 'P': '1'}).astype(float)
melt_df

,student,dates,attendence,attnd
0,Sally,2018-01-01,P,1.0
1,Jane,2018-01-01,A,0.0
2,Billy,2018-01-01,A,0.0
3,John,2018-01-01,P,1.0
4,Sally,2018-01-02,T,0.9
5,Jane,2018-01-02,P,1.0
6,Billy,2018-01-02,T,0.9
7,John,2018-01-02,T,0.9
8,Sally,2018-01-03,T,0.9
9,Jane,2018-01-03,T,0.9


In [32]:
#create a pivot table with students as the index and values as the combined attendance float values
grades = melt_df.pivot_table(index=['student'], values='attnd')
grades

,attnd
student,
Billy,0.5250
Jane,0.6875
John,0.9125
Sally,0.7625


In [69]:
#can also do a groupby
melt_df.groupby('student').attnd.mean()

student
Billy    0.5250
Jane     0.6875
John     0.9125
Sally    0.7625
Name: attnd, dtype: float64

---

2. Coffee Levels

- Read the coffee_levels table.
- Transform the data so that each carafe is in it's own column.
- Is this the best shape for the data?

In [33]:
#pull the coffee_levels dataset from the database
df = pd.read_sql('SELECT * FROM coffee_levels', get_db_url('tidy_data'))
#view the pulled dataset
df

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291
5,13,x,0.310711
6,14,x,0.507288
7,15,x,0.215043
8,16,x,0.183891
9,17,x,0.391560


In [37]:
#Transform the data so that each carafe is in it's own column
pivot = df.pivot_table(index=['hour'], columns='coffee_carafe', values='coffee_amount')
pivot

coffee_carafe,x,y,z
hour,,,
8,0.816164,0.189297,0.999264
9,0.451018,0.521502,0.915990
10,0.843279,0.023163,0.144928
11,0.335533,0.235529,0.311495
12,0.898291,0.017009,0.771947
13,0.310711,0.997464,0.398520
14,0.507288,0.058361,0.864464
15,0.215043,0.144644,0.436364
16,0.183891,0.544676,0.280621


In [70]:
#Is this the best shape for the data?
# -- It depends on what question you are trying to answer

---

3. Cake Recipes

- Read the cake_recipes table. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.
- Tidy the data as necessary.
- Which recipe, on average, is the best? recipe b
- Which oven temperature, on average, produces the best results? 275
- Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [39]:
#pull the cake_recipes dataset from the database
df = pd.read_sql('SELECT * FROM cake_recipes', get_db_url('tidy_data'))
#view the pulled dataset
df

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [41]:
#rename the 'recipe:position column' to replace the ':' with '_'
df.rename(columns={'recipe:position': 'recipe_position'}, inplace=True)
#review return df
df

,recipe_position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [43]:
#split the data in the recipe_position column into two columns 
df['recipe'] = df.recipe_position.str.split(':').str[0]
df['position'] = df.recipe_position.str.split(':').str[1]
#drop the original column
df = df.drop(columns='recipe_position')
#review the returned df
df

,225,250,275,300,recipe,position
0,61.738655,53.912627,74.414730,98.786784,a,bottom
1,51.709751,52.009735,68.576858,50.228470,a,top
2,57.095320,61.904369,61.196980,99.248541,b,bottom
3,82.455004,95.224151,98.594881,58.169349,b,top
4,96.470207,52.001358,92.893227,65.473084,c,bottom
5,71.306308,82.795477,92.098049,53.960273,c,top
6,52.799753,58.670419,51.747686,56.183110,d,bottom
7,96.873178,76.101363,59.571620,50.971626,d,top


In [48]:
#melt for score pivot table 
dfmelt = df.melt(id_vars=['recipe', 'position'], var_name='temps', value_name='score')
dfmelt

,recipe,position,temps,score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207
5,c,top,225,71.306308
6,d,bottom,225,52.799753
7,d,top,225,96.873178
8,a,bottom,250,53.912627
9,a,top,250,52.009735


In [49]:
#Which recipe, on average, is the best?
#score by recipe pivot table
pivot1 = dfmelt.pivot_table(index=['recipe'], values='score')
pivot1
#recipe b has the best average score

,score
recipe,
a,63.922201
b,76.736074
c,75.874748
d,62.864844


In [71]:
#can also do 
dfmelt.groupby('recipe').score.mean()

recipe
a    63.922201
b    76.736074
c    75.874748
d    62.864844
Name: score, dtype: float64

In [50]:
#Which oven temperature, on average, produces the best results?
#score by temp pivot table
pivot2 = dfmelt.pivot_table(index=['temps'], values='score')
pivot2
#best average - 275

,score
temps,
225,71.306022
250,66.577437
275,74.886754
300,66.627655


In [72]:
#can also do
dfmelt.groupby('temps').score.mean()

temps
225    71.306022
250    66.577437
275    74.886754
300    66.627655
Name: score, dtype: float64

In [52]:
#Which combination of recipe, rack position, and temperature gives the best result?
#pivot table - score by recipe, position, temps
pivot3 = dfmelt.pivot_table(index=['recipe', 'position', 'temps'], values='score').reset_index()
pivot3

,recipe,position,temps,score
0,a,bottom,225,61.738655
1,a,bottom,250,53.912627
2,a,bottom,275,74.414730
3,a,bottom,300,98.786784
4,a,top,225,51.709751
5,a,top,250,52.009735
6,a,top,275,68.576858
7,a,top,300,50.228470
8,b,bottom,225,57.095320
9,b,bottom,250,61.904369


In [56]:
#sort by score to see the highest first
pivot3.sort_values(by='score', ascending=False)

,recipe,position,temps,score
11,b,bottom,300,99.248541
3,a,bottom,300,98.786784
14,b,top,275,98.594881
28,d,top,225,96.873178
16,c,bottom,225,96.470207
13,b,top,250,95.224151
18,c,bottom,275,92.893227
22,c,top,275,92.098049
21,c,top,250,82.795477
12,b,top,225,82.455004


In [61]:
#highest combo score
pivot3.sort_values(by='score', ascending=False).head(1)

,recipe,position,temps,score
11,b,bottom,300,99.248541


In [62]:
#highest combo score another way
pivot3.nlargest(columns='score', n=1, keep='all')

,recipe,position,temps,score
11,b,bottom,300,99.248541


In [63]:
#didnt need to make the last pivot table because it used all the variables from the melt column. 
#could just have run it from the melt df
#the last pivot table would have been necessary if there were other variables in the df that I was not going to use.
dfmelt.nlargest(columns='score', n=1, keep='all')

,recipe,position,temps,score
26,b,bottom,300,99.248541


### Bonus: 
- explore the other tables in the tidy_data database and reshape them as necessary so that they are in a tidy format.

In [65]:
#pull the dataset from the database
df = pd.read_sql('SELECT * FROM students', get_db_url('tidy_data'))
#view the pulled dataset
df.head()

,date,var,val
0,2019-02-04,n_late_from_break,4.028120
1,2019-02-04,coffee_consumption,5255.409740
2,2019-02-04,classroom_temp,67.000000
3,2019-02-05,n_late_from_break,2.101998
4,2019-02-05,coffee_consumption,8603.704719


In [67]:
df.pivot_table(index=['date'], columns='var', values='val')

var,classroom_temp,coffee_consumption,n_late_from_break
date,,,
2019-02-04,67.0,5255.409740,4.028120
2019-02-05,73.0,8603.704719,2.101998
2019-02-06,81.0,1801.498050,4.941244
2019-02-07,62.0,9282.959741,1.419342
2019-02-08,72.0,7558.270659,1.808919
2019-02-11,60.0,5731.008713,4.831584
2019-02-12,52.0,9547.673484,3.550830
2019-02-13,56.0,7114.931847,2.550362
2019-02-14,72.0,1977.295513,3.507548


---

In [68]:
#pull the dataset from the database
df = pd.read_sql('SELECT * FROM billboard', get_db_url('tidy_data'))
#view the pulled dataset
df.head()

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,None,None,None,None,None,None,None,None,None,None
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,None,None,None,None,None,None,None,None,None,None
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,None,None,None,None,None,None,None,None,None,None
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,None,None,None,None,None,None,None,None,None,None
